In [1]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

2024-07-14 11:22:01.129018: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 11:22:01.129086: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 11:22:01.131173: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 11:22:01.345422: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-14 11:22:05.905451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 11:22:05.946848: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-14 11:22:05.946883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
import os


In [3]:
os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()


['run.sh',
 'requirements.txt',
 'ml',
 'convert_to_utf.py',
 '.idea',
 'bot_discord',
 'out',
 'venv',
 '.git',
 '.gitignore']

In [4]:
params_grid = {
    "embedder_data": "fasttext",
    "dataset": "corrected_format_all_data.csv"
}


In [5]:
path_folder = 'ml/ml_core/'
data_folder_src = 'ml/ml_core/data/processed/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'

vector_path = f'ml/ml_core/embedded_vector/{params_grid["embedder_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'


ft = fasttext.load_model(f'ml/ml_core/data/embedding_data/cc.fr.300.bin')


In [6]:

ds = pd.read_csv(data_path_src)
ds

,labels,text
0,1,rt @user personnage ainsi q tte la clique gauc...
1,1,@user @user @user bah oui t'as raison l'autre ...
2,1,moi j'ai personne à qui m'adresser pour réclam...
3,0,@user @user c...est vrai que les anchois sont ...
4,0,eh les renois faut se réveiller la @url
...,...,...
16893,0,L'islam a élevé la Femme en islam une femme es...
16894,0,L'islam considère que la femme est un joyau. E...
16895,0,L'islam valorise la femme sur beaucoup d'aspec...
16896,0,"Le Coran a libéré la femme, l'a honoré en tant..."


In [7]:

def text_to_vector(text):
    words = text.split(' ')
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    if not word_vectors:
        return np.zeros(300)
    return np.mean(word_vectors, axis=0)


In [8]:
tqdm.pandas(desc='Loading data')
ds['vector'] = ds['text'].progress_apply(text_to_vector)

X = np.vstack(ds['vector'])
y = ds['labels'].values


Loading data: 100%|██████████| 16898/16898 [13:26<00:00, 20.96it/s]


In [9]:
np.save(vector_x_dst, X)
np.save(vector_y_dst, y)